In [1]:
'''solution should involve creating a neuron compartment and actually running simulations 
   and calculating R_inp from the simulation output.
   Also, you should store the intermediates so that after optimization 
   you can plot tested parameters vs. tested R_inp, 
   and also tested parameters vs. tested objective error!'''
%matplotlib notebook
import scipy.optimize as opt
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from neuron import h
from neuron.units import ms, mV
import numpy as np
#from pylab import *; ion(); 

In [2]:
#set cell size, time vector, and soma Voltage vector ; add a leak channel 
soma = h.Section()
soma.L , soma.diam = 20. , 20.
seg = soma(0.5)
h.load_file('stdrun.hoc')
t = h.Vector()
soma_voltage = h.Vector()
t.record(h._ref_t, h.dt)  # record the time base
soma_voltage.record(soma(0.5)._ref_v, h.dt)
h.tstop = 600.
h.v_init = -65. 
#Insert a leak channel mechanism
soma.insert('pas')
leak_mechanism = soma(0.5).pas
pas_e0 = -70.
leak_mechanism.e = pas_e0
#leak_current_rec = h.Vector()
#leak_current_rec.record(soma(0.5).pas._ref_i, h.dt)

In [3]:
#Inject current through Patch Clamp and vectorize
step_current_stim = h.IClamp(soma(0.5))
step_current_stim.amp, step_current_stim.dur, step_current_stim.delay
step_current_stim.amp = 0.1  # amplitude in nanoAmps
step_current_stim.dur = 200.  # duration in milliseconds
step_current_stim.delay = 200.  # start time of current injection 
step_current_rec = h.Vector()
step_current_rec.record(step_current_stim._ref_i, h.dt)
h.run()

0.0

In [41]:
R_inp_list = []
pas_g0_list = []

#pas_g0 = [np.linspace(1e-12, 0.1, 30)]
pas_g0 = [0.00053]
#leak_mechanism.g = pas_g0
x = pas_g0
#h.run()
def R_inp_calc(x):
    soma(0.5).pas.g = x[0] 
    h.run()
    leak_current_rec = h.Vector()
    #leak_current_rec.record(soma(0.5).pas._ref_i, h.dt)
    soma_voltage_array = np.array(soma_voltage)
    t_array = np.array(t)
    step_current_amp = step_current_stim.amp
    baseline_eq_indexes = np.where((195. <= t_array) & (t_array < 200.))
    step_eq_indexes = np.where((395. <= t_array) & (t_array < 400.))
    baseline_eq_Vm  = np.mean(soma_voltage_array[baseline_eq_indexes])
    step_eq_Vm = np.mean(soma_voltage_array[step_eq_indexes])
    #print(baseline_eq_Vm, step_eq_Vm)
    delta_Vm = abs(baseline_eq_Vm - step_eq_Vm)
    R_inp = (delta_Vm / 1000.) / (abs(step_current_amp) / 1e9) / 1e6 # convert mV to V, nA to A, and Ohm to MegaOhm
    #print(R_inp)
    R_inp_list.append(R_inp)
    pas_g0_list.append(pas_g0) 
    return R_inp


In [42]:
#for trial in range(20):
R_inp_calc(x)

150.14617272819473

In [43]:
R_inp_list

[150.14617272819473]

In [44]:
pas_g0_list

[[0.00053]]

In [71]:
def f(x):
    return  abs(150 - R_inp_calc(x)) 

In [72]:
'''def objective(x):
    return -R_inp_calc(x) '''

'def objective(x):\n    return -R_inp_calc(x) '

In [73]:
'''def constraint(x):
    return  150 - R_inp_calc(x) '''

'def constraint(x):\n    return  150 - R_inp_calc(x) '

In [74]:
#cons = ({'type': 'ineq', 'fun': constraint})

In [75]:
#xguess, xguess2, xguess3 = 0.0005 , 0.005 , 0.00053

In [76]:
#x0 = np.array([xguess, xguess2, xguess3])

In [77]:
#sol = opt.minimize(objective,x0,method = 'SLSQP', constraints = cons, options ={'disp' : True})

In [78]:
#xOpt = sol.x

In [79]:
#xFun = sol.fun

In [80]:
#xOpt

In [81]:
#fun = R_inp_calc(xOpt) 
#fun

In [82]:
#x = np.linspace(1e-12, 0.01, 30)
#x

In [83]:
#y = f(x)
#for x in range(20):
    #print(y)y
#y

In [84]:
[xopt , fopt , info] = opt.fmin_l_bfgs_b(f, x0=[pas_g0], bounds = ((1e-12, 0.1),), 
                                         maxiter = 2500, maxfun = 2500, approx_grad=True, disp=True)

In [85]:
print(info)

{'grad': array([185807.13553717]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 120, 'nit': 2, 'warnflag': 2}


In [86]:
#[xopt , fopt , gopt , Bopt, func_calls, grad_calls , warnflag] = opt.fmin_bfgs(f, x0=[pas_g0], maxiter = 100, full_output = True)

In [87]:
xopt

array([0.00053051])

In [88]:
fopt

0.0004846635044088998

In [89]:
#xopt = opt.fsolve(f, x0)
#xopt

In [90]:
info['grad']

array([185807.13553717])

In [91]:
info['funcalls']

120

In [92]:
info['warnflag']

2

In [93]:
info['task']

b'ABNORMAL_TERMINATION_IN_LNSRCH'